In [1]:

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), r'C:\Users\Joel\Desktop\Sprints Case Study'))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils.connectors import API_Client
from utils.transformations import remove_invalid_rows, convert_to_dataframe, convert_currency, annual_aggregation
from datetime import datetime
import pandas as pd

base_url = "https://technical-case-platform-engineer.onrender.com"

exchange_rate_endpoint = "exchange-rates"
monthly_date_endpoint = "monthly-data"
annual_data_endpoint = "annual-data"

today = datetime.today()

In [2]:
api_connector = API_Client(base_url)

currency_data = api_connector.get(exchange_rate_endpoint)
value_data = api_connector.get(monthly_date_endpoint)

currency_df = convert_to_dataframe(currency_data)
value_df = convert_to_dataframe(value_data)

In [3]:
cleaned_value_df = remove_invalid_rows(value_df, currency_df)
cleaned_value_df = cleaned_value_df[cleaned_value_df["currency"]=="GBP"]
cleaned_value_df

,timestamp,value,company,currency


In [4]:
converted_df = convert_currency(cleaned_value_df, currency_df)
converted_df = converted_df[converted_df["currency"]=="GBP"]
converted_df

,timestamp,value,company,currency


In [5]:
annual_agg = annual_aggregation(converted_df)
annual_agg

,company,year,currency,value
